In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from src.data_loader import read_data

train_data = read_data('train')
val_data = read_data('val')
test_data = read_data('test')

In [ ]:
from src.config import BATCH_SIZE
import random 
import numpy as np 
import torch 

from src.networks.CLIP_Basic import CLIP_only_dataset

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

def get_dataloader(data):
    dataset = CLIP_only_dataset(data)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, worker_init_fn=seed_worker, generator=g)
    return data_loader
    
train_loader = get_dataloader(train_data)
val_loader = get_dataloader(val_data)
test_loader = get_dataloader(test_data)

In [ ]:
from src.trainer import Trainer
from src.networks.CLIP_Basic import CLIP_only_Model, CLIP_only_Input_transformer
import torch.nn as nn
import torch 

trainer = Trainer(10)

#Set data
trainer.set_data(train_loader, val_loader)

#set model 
model = CLIP_only_Model().cuda()
input_transformer = CLIP_only_Input_transformer()
trainer.set_model(model, input_transformer)

#Backpropagation
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
trainer.set_optimizer(optimizer, loss_fn)
trainer.train()

In [ ]:
trainer.test(test_loader)

In [ ]:
from captum.attr import LayerConductance
ig = LayerConductance(model, model.l1)
x, y= next(iter(test_loader)) 
x,y = input_transformer.transform(x,y)

In [ ]:
score = ig.attribute(inputs=x, target=1, n_steps=5)